In [ ]:
import pickle

In [ ]:
rets = {}

In [78]:
path = "cassie_0_z_pd_ret"
with open("{}.pkl".format(path), "rb+") as file:
    data = pickle.load(file)

In [79]:
# resort
for seed, epi_data in data.items():
    for obs_index, obs_ret in epi_data.items():
        if obs_index == "seed":
            continue
        resort = sorted(obs_ret, key=lambda x: x["neuron"]["neuron_index"], reverse=True)
        resort = sorted(resort, key=lambda x: x["neuron"]["layer"], reverse=True)
        epi_data[obs_index] = resort


In [80]:
# average
averaged_ret = {}
for seed, epi_data in data.items():
    for obs, obs_ret in epi_data.items():
        if obs == "seed":
            continue
        if obs in averaged_ret:
            for k,each_neuron in enumerate(obs_ret):
                assert averaged_ret[obs][k]["layer"] == each_neuron["neuron"]["layer"] and averaged_ret[obs][k]["neuron_index"] == each_neuron["neuron"]["neuron_index"], "{} {}".format(averaged_ret[obs][k]["neuron_index"], each_neuron["neuron"]["neuron_index"])
                averaged_ret[obs][k]["freq_diff"] = (averaged_ret[obs][k]["freq_diff"]*averaged_ret[obs][k]["seed_num"] + each_neuron["error"]["freq_diff"])/(averaged_ret[obs][k]["seed_num"]+1)
                averaged_ret[obs][k]["base_phase_diff"] = (averaged_ret[obs][k]["base_phase_diff"]*averaged_ret[obs][k]["seed_num"] + each_neuron["error"]["base_phase_diff"])/(averaged_ret[obs][k]["seed_num"]+1)
                averaged_ret[obs][k]["seed_num"]+=1
        else:
            averaged_ret[obs] = []
            for each_neuron in obs_ret:
                averaged_ret[obs].append({"layer": each_neuron["neuron"]["layer"],
                                   "neuron_index": each_neuron["neuron"]["neuron_index"],
                                   "freq_diff": each_neuron["error"]["freq_diff"],
                                   "base_phase_diff": each_neuron["error"]["base_phase_diff"],
                                   "seed_num": 1})
for obs_dim, avg_ret in averaged_ret.items():
    averaged_ret[obs_dim] = sorted(avg_ret, key=lambda x:x["freq_diff"],reverse=False)

rets[path] = averaged_ret

In [84]:
print(averaged_ret[4])

[{'layer': 0, 'neuron_index': 487, 'freq_diff': 34.50917537596307, 'base_phase_diff': 0.8571428310821496, 'seed_num': 1}, {'layer': 0, 'neuron_index': 84, 'freq_diff': 34.721586724047384, 'base_phase_diff': 0.9206349466956282, 'seed_num': 1}, {'layer': 0, 'neuron_index': 265, 'freq_diff': 34.81063483635908, 'base_phase_diff': 0.9682539421932607, 'seed_num': 1}, {'layer': 0, 'neuron_index': 88, 'freq_diff': 35.1823839156518, 'base_phase_diff': 0.8412698642386005, 'seed_num': 1}, {'layer': 0, 'neuron_index': 212, 'freq_diff': 35.71230884045645, 'base_phase_diff': 0.7936508197115011, 'seed_num': 1}, {'layer': 0, 'neuron_index': 15, 'freq_diff': 36.01593090854041, 'base_phase_diff': 0.8795418346463837, 'seed_num': 1}, {'layer': 0, 'neuron_index': 206, 'freq_diff': 36.11084413317209, 'base_phase_diff': 0.9206348945742131, 'seed_num': 1}, {'layer': 0, 'neuron_index': 385, 'freq_diff': 36.13790787222276, 'base_phase_diff': 0.9715237816029538, 'seed_num': 1}, {'layer': 0, 'neuron_index': 492, 

In [65]:

# 12-23 
# 24-35 497 230 40
print(rets["anymal_elu_pd_ret"][3])

[{'layer': 0, 'neuron_index': 143, 'freq_diff': 27.927851276689058, 'base_phase_diff': 0.5682539628651441, 'seed_num': 10}, {'layer': 0, 'neuron_index': 118, 'freq_diff': 29.373685453091618, 'base_phase_diff': 0.6609888410621163, 'seed_num': 10}, {'layer': 0, 'neuron_index': 206, 'freq_diff': 29.68908333073781, 'base_phase_diff': 0.6380952434840624, 'seed_num': 10}, {'layer': 0, 'neuron_index': 238, 'freq_diff': 30.122956248167878, 'base_phase_diff': 0.6300703191829399, 'seed_num': 10}, {'layer': 0, 'neuron_index': 374, 'freq_diff': 30.63742145497138, 'base_phase_diff': 0.658730153252993, 'seed_num': 10}, {'layer': 0, 'neuron_index': 74, 'freq_diff': 31.18028704520534, 'base_phase_diff': 0.5349206348322936, 'seed_num': 10}, {'layer': 0, 'neuron_index': 99, 'freq_diff': 31.67849234119646, 'base_phase_diff': 0.4936507939158178, 'seed_num': 10}, {'layer': 0, 'neuron_index': 252, 'freq_diff': 31.873349479708974, 'base_phase_diff': 0.7809589338606865, 'seed_num': 10}, {'layer': 0, 'neuron_i